In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import pickle
%matplotlib inline

In [ ]:
##CALIBRATION SECTION
images = glob.glob('./camera_cal/calibration*.jpg')

object_points = []
image_points = []

objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

#develop list of image points based on all picture of chess boards in camera_cal folder
for fname in images:
    img = mpimg.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    if ret == True:
        image_points.append(corners)
        object_points.append(objp)

        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)

# Test undistortion on an image
img = cv2.imread('./camera_cal/calibration3.jpg')
img_size = (img.shape[1], img.shape[0])

# Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, img_size, None, None)

dst = cv2.undistort(img, mtx, dist, None, mtx)
cv2.imwrite('./camera_cal/calibration2_undist.jpg', dst)

# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "camera_cal/undistort_pickle.p", "wb" ) )

#Show change on undistortion on example image
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=30)

In [ ]:
##DISTORTION CORRECTION SECTION
# Test undistortion on an image
img = mpimg.imread('./test_images/test5.jpg')
img_size = (img.shape[1], img.shape[0])

# Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, img_size, None, None)

undistorted_img = cv2.undistort(img, mtx, dist, None, mtx)
cv2.imwrite('./camera_cal/calibration3_undist.jpg', undistorted_img)

#Show change on undistortion on example image
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(undistorted_img)
ax2.set_title('Undistorted Image', fontsize=30)

In [ ]:
from matplotlib.path import Path
import matplotlib.patches as patches

def get_warp_matrix(img):
    img_size = (int(img.shape[1]/2), img.shape[0])
    bot_width = 1.1
    mid_width = .26
    height_pct = .665
    bottom_trim = .935
    src = np.float32([
            [img.shape[1]*(.5-mid_width/2), img.shape[0]*height_pct],
            [img.shape[1]*(.5+mid_width/2), img.shape[0]*height_pct], 
            [img.shape[1]*(.5+bot_width/2), img.shape[0]*bottom_trim], 
            [img.shape[1]*(.5-bot_width/2), img.shape[0]*bottom_trim]
    ])
    offset = img_size[0]*.15
    dst = np.float32([[offset, 0], [img_size[0] - offset, 0], [img_size[0] - offset, img_size[1]], [offset, img_size[1]]])
    
    M = cv2.getPerspectiveTransform(src,dst)
    M_inv = cv2.getPerspectiveTransform(dst, src)
    
    return M, M_inv

img = mpimg.imread('./camera_cal/calibration4_undist.jpg')
img_size = (int(img.shape[1]/2), img.shape[0])
M, M_inv = get_warp_matrix(img)
warped = cv2.warpPerspective(img, M, img_size,flags=cv2.INTER_LINEAR)

plt.imshow(img)
plt.show()
plt.imshow(warped)
plt.show()

In [ ]:
warped = cv2.warpPerspective(undistorted_img, M, img_size,flags=cv2.INTER_LINEAR)

plt.imshow(undistorted_img, cmap='gray')
plt.show()
plt.imshow(warped, cmap='gray')
plt.show()

In [ ]:
#Use color transforms, gradients, etc., to create a thresholded binary image.

def color_threshold(img):
    kernel = np.ones((14,14),np.uint8)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS) 
    
    hls_l = hls[:,:,1]
    th_hls_l = cv2.morphologyEx(hls_l, cv2.MORPH_TOPHAT, kernel)
    hls_l_binary = np.zeros_like(th_hls_l)
    hls_l_binary[(th_hls_l > 20) & (th_hls_l <= 255)] = 1
#     plt.imshow(hls_l_binary)
#     plt.show()
    
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB) 
    lab_l = lab[:,:,0]
    th_lab_l = cv2.morphologyEx(lab_l, cv2.MORPH_TOPHAT, kernel)
    lab_l_binary = np.zeros_like(th_lab_l)
    lab_l_binary[(th_lab_l > 20) & (th_lab_l <= 255)] = 1
#     plt.imshow(lab_l_binary)
#     plt.show()
    
    lab_b = lab[:,:,2]
    th_lab_b = cv2.morphologyEx(lab_b, cv2.MORPH_TOPHAT, kernel)
    lab_b_binary = np.zeros_like(th_lab_b)
    lab_b_binary[(th_lab_b > 5) & (th_lab_b <= 255)] = 1
#     plt.imshow(lab_b_binary)
#     plt.show()
    
    full_mask = np.zeros_like(th_hls_l)
    full_mask[(hls_l_binary == 1) | (lab_l_binary == 1) | (lab_b_binary == 1)] = 1
#     plt.imshow(full_mask)
#     plt.show()
    
    kernel = np.ones((6,3),np.uint8)
    erosion = cv2.erode(full_mask,kernel,iterations = 1)
#     plt.imshow(erosion)
#     plt.show()
    
    return erosion

def threshold_image(img):
    preprocessed_img = np.zeros_like(img[:,:,0])
    masked = color_threshold(img)
    
    return masked



thresholded_image = threshold_image(warped)
plt.imshow(thresholded_image, cmap='gray')
plt.show()


### Apply a perspective transform to rectify binary image ("birds-eye view").


In [ ]:
#Detect lane pixels and fit to find the lane boundary.
histogram = np.sum(thresholded_image[thresholded_image.shape[0]/2:,:], axis=0)
plt.plot(histogram)


In [ ]:
# Assuming you have created a warped binary image called "binary_warped"
# Take a histogram of the bottom half of the image
binary_warped = thresholded_image

# Create an output image to draw on and  visualize the result
out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
# Find the peak of the left and right halves of the histogram
# These will be the starting point for the left and right lines
midpoint = np.int(histogram.shape[0]/2)
leftx_base = np.argmax(histogram[:midpoint])
rightx_base = np.argmax(histogram[midpoint:]) + midpoint

# Choose the number of sliding windows
nwindows = 9
# Set height of windows
window_height = np.int(binary_warped.shape[0]/nwindows)
# Identify the x and y positions of all nonzero pixels in the image
nonzero = binary_warped.nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])
# Current positions to be updated for each window
leftx_current = leftx_base
rightx_current = rightx_base
# Set the width of the windows +/- margin
margin = 30
# Set minimum number of pixels found to recenter window
minpix = 50
# Create empty lists to receive left and right lane pixel indices
left_lane_inds = []
right_lane_inds = []

# Step through the windows one by one
for window in range(nwindows):
    # Identify window boundaries in x and y (and right and left)
    win_y_low = binary_warped.shape[0] - (window+1)*window_height
    win_y_high = binary_warped.shape[0] - window*window_height
    win_xleft_low = leftx_current - margin
    win_xleft_high = leftx_current + margin
    win_xright_low = rightx_current - margin
    win_xright_high = rightx_current + margin
    # Draw the windows on the visualization image
    cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
    cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
    # Identify the nonzero pixels in x and y within the window
    good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
    good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
    # Append these indices to the lists
    left_lane_inds.append(good_left_inds)
    right_lane_inds.append(good_right_inds)
    # If you found > minpix pixels, recenter next window on their mean position
    if len(good_left_inds) > minpix:
        leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
    if len(good_right_inds) > minpix:        
        rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

# Concatenate the arrays of indices
left_lane_inds = np.concatenate(left_lane_inds)
right_lane_inds = np.concatenate(right_lane_inds)

# Extract left and right line pixel positions
leftx = nonzerox[left_lane_inds]
lefty = nonzeroy[left_lane_inds] 
rightx = nonzerox[right_lane_inds]
righty = nonzeroy[right_lane_inds] 

# Fit a second order polynomial to each
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)

# Generate x and y values for plotting
ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
plt.imshow(out_img)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 640)
plt.ylim(720, 0)

In [ ]:
# Assume you now have a new warped binary image 
# from the next frame of video (also called "binary_warped")
# It's now much easier to find line pixels!
nonzero = binary_warped.nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])
margin = 30
left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

# Again, extract left and right line pixel positions
leftx = nonzerox[left_lane_inds]
lefty = nonzeroy[left_lane_inds] 
rightx = nonzerox[right_lane_inds]
righty = nonzeroy[right_lane_inds]
# Fit a second order polynomial to each
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)
# Generate x and y values for plotting
ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

In [ ]:
# Create an image to draw on and an image to show the selection window
out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
window_img = np.zeros_like(out_img)
# Color in left and right line pixels
out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

# Generate a polygon to illustrate the search window area
# And recast the x and y points into usable format for cv2.fillPoly()
left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
left_line_pts = np.hstack((left_line_window1, left_line_window2))
right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
right_line_pts = np.hstack((right_line_window1, right_line_window2))

# Draw the lane onto the warped blank image
cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
plt.imshow(result)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 640)
plt.ylim(720, 0)




In [ ]:
# Generate some fake data to represent lane-line pixels
ploty = np.linspace(0, 719, num=720)# to cover same y-range as image
quadratic_coeff = 3e-4 # arbitrary quadratic coefficient
# For each y position generate random x position within +/-50 pix
# of the line base position in each case (x=200 for left, and x=900 for right)
leftx = left_fitx
rightx = right_fitx

# Fit a second order polynomial to pixel positions in each fake lane line
left_fit = np.polyfit(ploty, leftx, 2)
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fit = np.polyfit(ploty, rightx, 2)
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

# Plot up the fake data
mark_size = 3
plt.plot(leftx, ploty, 'o', color='red', markersize=mark_size)
plt.plot(rightx, ploty, 'o', color='blue', markersize=mark_size)
plt.xlim(0, 640)
plt.ylim(0, 720)
plt.plot(left_fitx, ploty, color='green', linewidth=3)
plt.plot(right_fitx, ploty, color='green', linewidth=3)
plt.gca().invert_yaxis() # to visualize as we do the images

In [ ]:
# Define y-value where we want radius of curvature
# I'll choose the maximum y-value, corresponding to the bottom of the image
ploty = np.linspace(0, 719, num=720)
y_eval = np.max(ploty)
left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
# Example values: 1926.74 1908.48
print(left_curverad)
print(right_curverad)

In [ ]:
# Define conversions in x and y from pixels space to meters
# 170 pixels = 3 meters in real world y axis
# 210 pixels = 3.7
ym_per_pix = 3/170 # meters per pixel in y dimension
xm_per_pix = 3.7/210 # meters per pixel in x dimension

# Fit new polynomials to x,y in world space
left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
# Calculate the new radii of curvature
left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
# Now our radius of curvature is in meters
road_curve = left_curverad + right_curverad / 2
road_curve = round(road_curve, 2)
curve_string = "{:,}".format(road_curve)
# Example values: 632.1 m    626.2 m

center_of_lines = (right_fitx[len(right_fitx)-1] + left_fitx[len(left_fitx)-1]) / 2
distance_from_center = center_of_lines - img.shape[1] / 4
distance_from_center = distance_from_center * xm_per_pix
distance_from_center = round(distance_from_center, 2)
center_offset = "{:,}".format(distance_from_center)

In [ ]:
# Create an image to draw the lines on
warp_zero = np.zeros_like(warped).astype(np.uint8)

# Recast the x and y points into usable format for cv2.fillPoly()
pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
pts = np.hstack((pts_left, pts_right))

# Draw the lane onto the warped blank image

cv2.fillPoly(warp_zero, np.int_([pts]), (0,255, 0))

img_size = (int(img.shape[1]), img.shape[0])
# Warp the blank back to original image space using inverse perspective matrix (Minv) 
newwarp = cv2.warpPerspective(warp_zero, M_inv, img_size,flags=cv2.INTER_LINEAR)
# Combine the result with the original image
print(newwarp.shape)
result = cv2.addWeighted(undistorted_img, 1, newwarp, 0.3, 0)

# Create a black image
img = np.zeros_like(result)

# Write some Text
font = cv2.FONT_HERSHEY_SIMPLEX
image = cv2.putText(result,'Road Curve: ' + curve_string + ' Meters',(10,50), font, 1.4,(255,255,255),2)
image = cv2.putText(image, 'Off Center: ' + center_offset + ' Meters',(10,100), font, 1.4,(255,255,255),2)

#Display the image

plt.imshow(image)



In [147]:
class Line():
    def __init__(self):
        self.recent_amount = 5
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None   
        #fit coefficients of the last n fits of the line
        self.recent_fits = []
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
      
            
    def confirm(self, target, compare):
        ret_value = True
        if compare is not None:
            if target < .5 * compare or target > 1.5 * compare:
                ret_value = False
        return ret_value
    
    def confirm_radius(self, target, compare):
        ret_value = True
        if compare is not None:
            if (target < compare / 5) | (target > 5 * compare):
                ret_value = False
        return ret_value
    
    def get_slope(self, fit):
        x1 = fit[len(fit)-1]
        y1 = len(fit)-1
        x2 = fit[0]
        y2 = 0
        m = (y2 - y1) / (x2 - x1)
        return m
    
    def sanity_check(self, new_radius, new_fitx):
        radius_similar = self.confirm_radius(new_radius, self.radius_of_curvature)
        #slope_similar = self.confirm(self.get_slope(new_fitx), self.get_slope(self.allx))
        line_position_similar = self.confirm(new_fitx[len(new_fitx)-1], self.bestx[len(self.bestx)-1])
        #return radius_similar and line_position_similar
        return radius_similar and line_position_similar
    
    def should_update(self, new_radius, new_fitx):
        should_update = False
        if self.detected:
            if self.sanity_check(new_radius, new_fitx):
                should_update = True
        else:
            should_update = True
        return should_update
    
    def get_radius(line):
        # Define y-value where we want radius of curvature
        # I'll choose the maximum y-value, corresponding to the bottom of the image
        y_eval = 720
        curverad = ((1 + (2*line[0]*y_eval + line[1])**2)**1.5) / np.absolute(2*line[0])
        # Example values: 1926.74 1908.48

        # Define conversions in x and y from pixels space to meters
        ym_per_pix = 30/720 # meters per pixel in y dimension
        xm_per_pix = 3.7/700 # meters per pixel in x dimension

        # Fit new polynomials to x,y in world space
        fit_cr = np.polyfit(ploty*ym_per_pix, line*xm_per_pix, 2)
        # Calculate the new radii of curvature
        curverad = ((1 + (2*fit_cr[0]*y_eval*ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])

        return curverad
    
    def update(self, new_fit, new_radius, new_fitx):
        if(self.should_update(new_radius, new_fitx)):
            self.detected = True
            self.diffs = self.current_fit - new_fit
            self.current_fit = new_fit
            self.recent_fits.append(self.current_fit)
            self.recent_fits = self.recent_fits[-self.recent_amount:]
            self.best_fit = np.mean(self.recent_fits, axis=0).astype(int)
            self.recent_xfitted.append(new_fitx)
            self.recent_xfitted = self.recent_xfitted[-self.recent_amount:]
            self.bestx = np.mean(self.recent_xfitted, axis=0).astype(int)
            self.allx = new_fitx
            self.radius_of_curvature = new_radius
        else:
            self.detected = False


In [212]:
def camera_calibation():
    images = glob.glob('./camera_cal/calibration*.jpg')

    object_points = []
    image_points = []

    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    #develop list of image points based on all picture of chess boards in camera_cal folder
    for fname in images:
        img = mpimg.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
        if ret == True:
            image_points.append(corners)
            object_points.append(objp)

            img = cv2.drawChessboardCorners(img, (9,6), corners, ret)

    # Test undistortion on an image
    img = cv2.imread('./camera_cal/calibration3.jpg')
    img_size = (img.shape[1], img.shape[0])

    # Do camera calibration given object points and image points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, img_size, None, None)
    return (mtx, dist)

def undistort(img, mtx, dist):
    undistorted_img = cv2.undistort(img, mtx, dist, None, mtx)
    return undistorted_img

#Use color transforms, gradients, etc., to create a thresholded binary image.
def get_warp_matrix(img):
    img_size = (int(img.shape[1]/2), img.shape[0])
    bot_width = 1.3
    mid_width = .30
    height_pct = .665
    bottom_trim = .945
    src = np.float32([
            [img.shape[1]*(.5-mid_width/2), img.shape[0]*height_pct],
            [img.shape[1]*(.5+mid_width/2), img.shape[0]*height_pct], 
            [img.shape[1]*(.5+bot_width/2), img.shape[0]*bottom_trim], 
            [img.shape[1]*(.5-bot_width/2), img.shape[0]*bottom_trim]
    ])
    offset = img_size[0]*.15
    dst = np.float32([[offset, 0], [img_size[0] - offset, 0], [img_size[0] - offset, img_size[1]], [offset, img_size[1]]])
    
    M = cv2.getPerspectiveTransform(src,dst)
    M_inv = cv2.getPerspectiveTransform(dst, src)
    
    return M, M_inv

def color_threshold(img):
    kernel = np.ones((14,14),np.uint8)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS) 
    
    hls_l = hls[:,:,1]
    th_hls_l = cv2.morphologyEx(hls_l, cv2.MORPH_TOPHAT, kernel)
    hls_l_binary = np.zeros_like(th_hls_l)
    hls_l_binary[(th_hls_l > 20) & (th_hls_l <= 255)] = 1

    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB) 
    lab_l = lab[:,:,0]
    th_lab_l = cv2.morphologyEx(lab_l, cv2.MORPH_TOPHAT, kernel)
    lab_l_binary = np.zeros_like(th_lab_l)
    lab_l_binary[(th_lab_l > 20) & (th_lab_l <= 255)] = 1

    lab_b = lab[:,:,2]
    th_lab_b = cv2.morphologyEx(lab_b, cv2.MORPH_TOPHAT, kernel)
    lab_b_binary = np.zeros_like(th_lab_b)
    lab_b_binary[(th_lab_b > 5) & (th_lab_b <= 255)] = 1

    full_mask = np.zeros_like(th_hls_l)
    full_mask[(hls_l_binary == 1) | (lab_l_binary == 1) | (lab_b_binary == 1)] = 1

    kernel = np.ones((6,3),np.uint8)
    erosion = cv2.erode(full_mask,kernel,iterations = 1)

    return erosion

def threshold_image(img):
    preprocessed_img = np.zeros_like(img[:,:,0])
    masked = color_threshold(img)
    return masked


def locate_line(thresholded, lane_side):
    histogram = np.sum(thresholded[thresholded.shape[0]/2:,:], axis=0)
    out_img = np.dstack((thresholded, thresholded, thresholded))*255
    midpoint = np.int(histogram.shape[0]/2)
    if lane_side == 'left':
        x_base = np.argmax(histogram[:midpoint])
    else:
        x_base = np.argmax(histogram[midpoint:]) + midpoint
    
    nwindows = 9
    window_height = np.int(thresholded.shape[0]/nwindows)
    nonzero = thresholded.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    x_current = x_base
    margin = 30
    minpix = 50
    lane_inds = []

    
    for window in range(nwindows):
        win_y_low = thresholded.shape[0] - (window+1)*window_height
        win_y_high = thresholded.shape[0] - window*window_height
        win_x_low = x_current - margin
        win_x_high = x_current + margin
        cv2.rectangle(out_img,(win_x_low,win_y_low),(win_x_high,win_y_high),(0,255,0), 2)
        good_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_x_low) & (nonzerox < win_x_high)).nonzero()[0]
        lane_inds.append(good_inds)
        if len(good_inds) > minpix:
            x_current = np.int(np.mean(nonzerox[good_inds]))
            
    lane_inds = np.concatenate(lane_inds)

    x = nonzerox[lane_inds]
    y = nonzeroy[lane_inds]
    if x.shape[0] != 0:
        line_fit = np.polyfit(y, x, 2)

        # Generate x and y values for plotting
        ploty = np.linspace(0, thresholded.shape[0]-1, thresholded.shape[0] )
        fitx = line_fit[0]*ploty**2 + line_fit[1]*ploty + line_fit[2]

        out_img[nonzeroy[lane_inds], nonzerox[lane_inds]] = [255, 0, 0]
    #     plt.imshow(out_img)
    #     plt.plot(fitx, ploty, color='yellow')
    #     plt.xlim(0, 640)
    #     plt.ylim(720, 0)
    #     plt.show()
        radius = get_radius(fitx, ploty)
    return line_fit, radius, fitx

def line_in_windows(thresholded, line):
    line_fit = fitx = radius = None
    line = line.current_fit
    out_img = np.dstack((thresholded, thresholded, thresholded))*255
    nonzero = thresholded.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 30
    lane_inds = ((nonzerox > (line[0]*(nonzeroy**2) + line[1]*nonzeroy + line[2] - margin)) & (nonzerox < (line[0]*(nonzeroy**2) + line[1]*nonzeroy + line[2] + margin))) 
    
    x = nonzerox[lane_inds]
    y = nonzeroy[lane_inds] 
    
    if x.shape[0] != 0:
        line_fit = np.polyfit(y, x, 2)
        ploty = np.linspace(0, thresholded.shape[0]-1, thresholded.shape[0] )
        fitx = line_fit[0]*ploty**2 + line_fit[1]*ploty + line_fit[2]
#         out_img[nonzeroy[lane_inds], nonzerox[lane_inds]] = [255, 0, 0]
#         plt.imshow(out_img)
#         plt.plot(fitx, ploty, color='yellow')
#         plt.xlim(0, 640)
#         plt.ylim(720, 0)
#         plt.show()
        radius = get_radius(fitx, ploty)
    
    return line_fit, radius, fitx   

def get_radius(line, ploty):
    # Define y-value where we want radius of curvature
    # I'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    curverad = ((1 + (2*line[0]*y_eval + line[1])**2)**1.5) / np.absolute(2*line[0])
    # Example values: 1926.74 1908.48
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 3/170 # meters per pixel in y dimension
    xm_per_pix = 3.7/210 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    fit_cr = np.polyfit(ploty*ym_per_pix, line*xm_per_pix, 2)
    # Calculate the new radii of curvature
    curverad = ((1 + (2*fit_cr[0]*y_eval*ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])

    return curverad
    
def overlay(warped, lines):
    if lines is not None:
        ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
        # Create an image to draw the lines on
        warp_zero = np.zeros_like(warped).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

        # Recast the x and y points into usable format for cv2.fillPoly()

        pts_left = np.array([np.transpose(np.vstack([lines['left'].bestx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([lines['right'].bestx, ploty])))])
        pts = np.hstack((pts_left, pts_right))

        # Draw the lane onto the warped blank image
        overlayer = cv2.fillPoly(warp_zero, np.int_([pts]), (0,255, 0))

    return overlayer

def get_line_fit(thresholded_image, lines, side):
    #print(lines[side].detected)
    line_fit = radius = fitx = None
    if lines[side].detected == True:
        line_fit, radius, fitx = line_in_windows(thresholded_image, lines[side])
    
    if line_fit == None:
        line_fit, radius, fitx = locate_line(thresholded_image, side)
    return line_fit, radius, fitx

def confirm_radius(target_radius, compare_radius):
    ret_value = True
    #print(target_radius)
    #print(compare_radius)
    if target_radius <  compare_radius / 5 or target_radius > 5 * compare_radius:
        ret_value = False
    return ret_value

def get_distance_between(target, compare):
    pixel_distance = target - compare
    xm_per_pix = 3.7/210 # meters per pixel in x dimension
    result = pixel_distance * xm_per_pix
    return result

def confirm_spacing(target_fit, compare_fit):
    ret_value = True
    gap_top = get_distance_between(target_fit[0], compare_fit[0])
    gap_bottom = get_distance_between(target_fit[len(target_fit)-1], compare_fit[len(compare_fit)-1])
    if abs(gap_top) > 4.1 or abs(gap_top) < 3.4:
        ret_value = False
        
    if abs(gap_bottom) > 4.1 or abs(gap_bottom) < 3.4:
        ret_value = False
        
    return ret_value

def confirm_position(target_fit, compare_fit):
    ret_value = True
    line_top = get_distance_between(target_fit[0], compare_fit[0])
    line_bottom = get_distance_between(target_fit[len(target_fit)-1], compare_fit[len(compare_fit)-1])
    if abs(line_top) > .1 or abs(line_top) < -.1:
        ret_value = False
        
    if abs(line_bottom) > .3 or abs(line_bottom) < -.3:
        ret_value = False
        
    return ret_value

def get_fit_slope(fit):
    x2 = fit[0]*720**2 + fit[1]*720 + fit[2]
    y2 = 720
    x1 = fit[0]*0**2 + fit[1]*0 + fit[2]
    y1 = 0
    m = (y2 - y1) / (x2 - x1)
    return m

def confirm_slope(target_fit, compare_fit):
    ret_value = True
    target_slope = get_fit_slope(target_fit)
    compare_slope = get_fit_slope(compare_fit)
    if target_slope <  compare_slope / 5 or target_slope > 5 * compare_slope:
        ret_value = False
    return True

def confirm_real_fit(target_fit, compare_fit, target_radius, compare_radius, line):
    ret_fit = line.current_fit
    radius_confirmation = confirm_radius_similarity(target_radius, compare_radius, line)
    lane_space_confirmation = get_line_space_confirmation(target_fit, compare_fit)
    parallel_confirmation = get_parallel_confirmation(target_fit, compare_fit, line)
    
    if radius_confirmation and lane_space_confirmation and parallel_confirmation:
        ret_fit = target_fit
    
    return ret_fit

def get_left_right_compliance(l_fit, l_radius, l_fitx, r_fit, r_radius, r_fitx):
    radius_similar = confirm_radius(l_radius, r_radius)
    slope_similar = confirm_slope(l_fit, r_fit)
    line_position_similar = confirm_spacing(l_fitx, r_fitx)
    return radius_similar and slope_similar and line_position_similar

def get_previous_compliance(fit1, radius1, fitx1, fit2, radius2, fitx2):
    radius_similar = confirm_radius(radius1, radius2)
    slope_similar = confirm_slope(fit1, fit2)
    line_position_similar = confirm_position(fitx1, fitx2)
    return radius_similar and slope_similar and line_position_similar

def pipeline(img):
    global lines, calibration
    undistorted_img = undistort(img, calibration[0], calibration[1])
    
    size_for_warp = (int(undistorted_img.shape[1]/2), undistorted_img.shape[0])
    M, M_inv = get_warp_matrix(undistorted_img)
    warped = cv2.warpPerspective(undistorted_img, M, size_for_warp, flags=cv2.INTER_LINEAR)
#     plt.imshow(warped)
#     plt.show()
    thresholded_image = threshold_image(warped)
#     plt.imshow(thresholded_image)
#     plt.show()
    #check if line has already been detected, if so use windowed method, if not start anew.
    
    
    l_fit, l_radius, l_fitx = get_line_fit(thresholded_image, lines, 'left')
    r_fit, r_radius, r_fitx = get_line_fit(thresholded_image, lines, 'right')
    if l_fit is not None and r_fit is not None:
        if get_left_right_compliance(l_fit, l_radius, l_fitx, r_fit, r_radius, r_fitx):
            lines['left'].update(l_fit, l_radius, l_fitx)
            lines['right'].update(r_fit, r_radius, r_fitx)
        else:
            if lines['left'].allx is None and lines['right'].allx is None:
                lines['left'].update(l_fit, l_radius, l_fitx)
                lines['right'].update(r_fit, r_radius, r_fitx)
            else:
                if lines['left'].allx is not None:
                    line = lines['left']
                    if get_previous_compliance(l_fit, l_radius, l_fitx, line.current_fit, line.radius_of_curvature, line.allx):
                        lines['left'].update(l_fit, l_radius, l_fitx)

                if lines['right'].allx is not None:
                    line = lines['right']
                    if get_previous_compliance(r_fit, r_radius, r_fitx, line.current_fit, line.radius_of_curvature, line.allx):
                        lines['right'].update(r_fit, r_radius, r_fitx)
            
    
    
        overlayer = overlay(warped, lines)
    else:
        lines['left'].detected = False
        lines['right'].detected = False
        l_fit, l_radius, l_fitx = get_line_fit(thresholded_image, lines, 'left')
        r_fit, r_radius, r_fitx = get_line_fit(thresholded_image, lines, 'right')
        if l_fit is not None and r_fit is not None:
            if get_left_right_compliance(l_fit, l_radius, l_fitx, r_fit, r_radius, r_fitx):
                lines['left'].update(l_fit, l_radius, l_fitx)
                lines['right'].update(r_fit, r_radius, r_fitx)
            else:
                if lines['left'].allx is None and lines['right'].allx is None:
                    lines['left'].update(l_fit, l_radius, l_fitx)
                    lines['right'].update(r_fit, r_radius, r_fitx)
                else:
                    if lines['left'].allx is not None:
                        line = lines['left']
                        if get_previous_compliance(l_fit, l_radius, l_fitx, line.current_fit, line.radius_of_curvature, line.allx):
                            lines['left'].update(l_fit, l_radius, l_fitx)

                    if lines['right'].allx is not None:
                        line = lines['right']
                        if get_previous_compliance(r_fit, r_radius, r_fitx, line.current_fit, line.radius_of_curvature, line.allx):
                            lines['right'].update(r_fit, r_radius, r_fitx)
            overlayer = overlay(warped, lines)
        else:
            overlayer = warped
            
        
    size_for_unwarp = (int(undistorted_img.shape[1]), undistorted_img.shape[0])
    unwarped = cv2.warpPerspective(overlayer, M_inv, size_for_unwarp, flags=cv2.INTER_LINEAR)
    overlayed = cv2.addWeighted(undistorted_img, 1, unwarped, 0.3, 0)
    
    
    ploty = np.linspace(0, 719, num=720)
    y_eval = np.max(ploty)
    img = np.zeros_like(undistorted_img)
    
    ym_per_pix = 3/170 # meters per pixel in y dimension
    xm_per_pix = 3.7/210 # meters per pixel in x dimension
    leftx = lines['left'].bestx
    rightx = lines['right'].bestx
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    road_curve = left_curverad + right_curverad / 2
    road_curve = round(road_curve, 2)
    curve_string = "{:,}".format(road_curve)
    if road_curve > 3000:
        curve_string = 'Straight'
    else:
        curve_string = str(curve_string) + 'm'
    # Example values: 632.1 m    626.2 m

    center_of_lines = (rightx[len(rightx)-1] + leftx[len(leftx)-1]) / 2
    distance_from_center = center_of_lines - img.shape[1] / 4
    distance_from_center = distance_from_center * xm_per_pix
    distance_from_center = round(distance_from_center, 2)
    center_offset = "{:,}".format(abs(distance_from_center))
    
    if distance_from_center > 0:
        center_offset = str(center_offset) + 'm left'
    else:
        center_offset = str(center_offset) + 'm right'

    # Write some Text
    font = cv2.FONT_HERSHEY_SIMPLEX
    image = cv2.putText(overlayed,'Road Curve: ' + curve_string,(10,50), font, 1.4,(255,255,255),2)
    image = cv2.putText(image, 'Off Center: ' + center_offset,(10,100), font, 1.4,(255,255,255),2)

    return image



In [122]:
global calibration
calibration = camera_calibation()

In [213]:
left_line = Line()
right_line = Line()

lines = {
    'left': left_line, 
    'right': right_line
}

images = glob.glob('./test_images/test*.jpg')

for idx, fname in enumerate(images):
#    print(fname)
    img = mpimg.imread(fname)
#     img = mpimg.imread('./test_images/test5.jpg')
    overlayed_img = pipeline(img)
#     plt.imshow(overlayed_img)
#     plt.show()

In [214]:
left_line = Line()
right_line = Line()

global lines
lines = {
    'left': left_line, 
    'right': right_line
}

In [215]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [200]:
first_output = 'first.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(pipeline)
white_clip.write_videofile(first_output, audio=False)

  self.nchannels))




[MoviePy] >>>> Building video first.mp4
[MoviePy] Writing video first.mp4








  0%|          | 0/1261 [00:00<?, ?it/s]WARNING:py.warnings:/Users/peternemrow/anaconda3/envs/sdc_env/lib/python3.5/site-packages/ipykernel/__main__.py:208: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.







  0%|          | 1/1261 [00:00<02:59,  7.02it/s]





  0%|          | 2/1261 [00:00<02:49,  7.41it/s]





  0%|          | 3/1261 [00:00<02:39,  7.91it/s]





  0%|          | 4/1261 [00:00<02:29,  8.41it/s]





  0%|          | 6/1261 [00:00<02:18,  9.07it/s]





  1%|          | 8/1261 [00:00<02:12,  9.44it/s]





  1%|          | 10/1261 [00:01<02:09,  9.68it/s]





  1%|          | 12/1261 [00:01<02:04, 10.00it/s]





  1%|          | 14/1261 [00:01<02:01, 10.25it/s]





  1%|▏         | 16/1261 [00:01<01:59, 10.38it/s]





  1%|▏         | 18/1261 [00:01<01:58, 10.49it/s]





  2%|▏         | 20/1261 [00:01<01:57, 10.53it/s]





  2%|▏         | 22/1261 [00:02<02:02, 10.15it/s]





  2%|▏         | 24/12

[MoviePy] Done.
[MoviePy] >>>> Video ready: first.mp4 



In [201]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(first_output))

In [216]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

challenge_output = 'challenge.mp4'
clip1 = VideoFileClip("challenge_video.mp4")
white_clip = clip1.fl_image(pipeline)
white_clip.write_videofile(challenge_output, audio=False)

  self.nchannels))




[MoviePy] >>>> Building video challenge.mp4
[MoviePy] Writing video challenge.mp4









  0%|          | 0/485 [00:00<?, ?it/s]WARNING:py.warnings:/Users/peternemrow/anaconda3/envs/sdc_env/lib/python3.5/site-packages/ipykernel/__main__.py:208: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.








  0%|          | 1/485 [00:00<01:00,  8.05it/s]






  0%|          | 2/485 [00:00<00:58,  8.30it/s]






  1%|          | 3/485 [00:00<00:56,  8.56it/s]






  1%|          | 4/485 [00:00<00:54,  8.81it/s]






  1%|          | 6/485 [00:00<00:51,  9.31it/s]






  2%|▏         | 8/485 [00:00<00:48,  9.77it/s]






  2%|▏         | 10/485 [00:00<00:46, 10.15it/s]






  2%|▏         | 12/485 [00:01<00:45, 10.48it/s]






  3%|▎         | 14/485 [00:01<00:44, 10.69it/s]






  3%|▎         | 16/485 [00:01<00:43, 10.89it/s]






  4%|▎         | 18/485 [00:01<00:42, 10.95it/s]






  4%|▍         | 20/485 [00:01<00:42, 10.98it/s]






  5%|▍         | 22/485 [00:02<00:41, 11.12it/s]






  5%|▍         | 24/4

[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge.mp4 



In [217]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

In [210]:
harder_challenge_output = 'harder_challenge_video_output.mp4'
clip1 = VideoFileClip("harder_challenge_video.mp4")
white_clip = clip1.fl_image(pipeline)
white_clip.write_videofile(harder_challenge_output, audio=False)

  self.nchannels))




[MoviePy] >>>> Building video harder_challenge_video_output.mp4
[MoviePy] Writing video harder_challenge_video_output.mp4









  0%|          | 0/1200 [00:00<?, ?it/s]WARNING:py.warnings:/Users/peternemrow/anaconda3/envs/sdc_env/lib/python3.5/site-packages/ipykernel/__main__.py:208: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.








  0%|          | 1/1200 [00:00<02:29,  8.00it/s]






  0%|          | 2/1200 [00:00<02:27,  8.14it/s]






  0%|          | 3/1200 [00:00<02:20,  8.54it/s]






  0%|          | 4/1200 [00:00<02:17,  8.68it/s]






  0%|          | 5/1200 [00:00<02:14,  8.90it/s]






  0%|          | 6/1200 [00:00<02:11,  9.08it/s]






  1%|          | 7/1200 [00:00<02:13,  8.93it/s]






  1%|          | 8/1200 [00:00<02:18,  8.60it/s]






  1%|          | 9/1200 [00:01<02:19,  8.53it/s]WARNING:py.warnings:/Users/peternemrow/anaconda3/envs/sdc_env/lib/python3.5/site-packages/ipykernel/__main__.py:89: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future








  1

[MoviePy] Done.
[MoviePy] >>>> Video ready: harder_challenge_video_output.mp4 



In [211]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(harder_challenge_output))